In [2]:
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [8]:
import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
%matplotlib inline

In [5]:
path = "/home/edwin/Datasets/competitions/landmark-recognition-challenge"

In [6]:
train_path = f"{path}/train.csv"

In [ ]:
test_path = f"{path}/test.csv"

In [ ]:
test_labels = pd.read_csv(test_path)

In [9]:
labels = pd.read_csv(train_path)

In [ ]:
test_path = "/home/edwin/Datasets/competitions/landmark-recognition-challenge/test"

In [ ]:
len(test_labels)

In [ ]:
def download_image(dataframe):
    for index, row in dataframe.iterrows():
        image_id = index
        filepath = f"{test_path}/{row['id']}.jpg"
        if dpath.exists(filepath):
            print("File already exists")
        else:
            ! wget -O "{filepath}" "{row['url']}"

In [ ]:
index, row = next(test_labels.iterrows())

In [ ]:
download_path

In [ ]:
row['id']

In [11]:
sample = labels.sample(frac=0.001)

In [12]:
train, valid = train_test_split(sample, train_size=0.8, test_size=0.2, random_state=42)

In [ ]:
train, valid

In [ ]:
train.shape, valid.shape

In [ ]:
len(set(labels.landmark_id.values))

In [13]:
INPUT_SIZE = 224
NUM_CLASSES = 14951

In [14]:
data_dir = "/home/edwin/Datasets/competitions/landmark-recognition-challenge/data/"

In [30]:
class LandmarkDataset(Dataset):
    def __init__(self, labels, root_dir, transform=None):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = '{}.jpg'.format(self.labels['id'].iloc[idx])
        fullname = join(self.root_dir, img_name)
        try:
            image = Image.open(fullname).convert('RGB')
        except:
            img_zeros = np.zeros([224, 224, 3])
            image = Image.fromarray(img_zeros.astype('uint8'), 'RGB')
        label = self.labels['landmark_id'].iloc[idx]
        if self.transform:
            image = self.transform(image)
        return [image, label]

In [ ]:
class LandmarkTestDataset(Dataset):
    def __init__(self, labels, root_dir, transform=None):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = '{}.jpg'.format(self.labels['id'].iloc[idx])
        fullname = join(self.root_dir, img_name)
        try:
            image = Image.open(fullname)
        except:
            img_zeros = np.zeros([224, 224, 3])
            image = Image.fromarray(img_zeros.astype('uint8'), 'RGB')
        label = self.labels['landmark_id'].iloc[idx]
        if self.transform:
            image = self.transform(image)
        return [image, label]

In [16]:
ds_transform = transforms.Compose([transforms.Resize(224),
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor()
                                 ])

In [17]:
train_ds = LandmarkDataset(train, data_dir, transform=ds_transform)
valid_ds = LandmarkDataset(valid, data_dir, transform=ds_transform)

train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=4)
valid_dl = DataLoader(valid_ds, batch_size=4, shuffle=True, num_workers=4)

In [ ]:
def imshow(axis, inp):
    inp = inp.numpy().transpose((1, 2, 0))
    axis.imshow(inp)

In [ ]:
img, label = next(iter(train_dl))
fig = plt.figure(1, figsize=(16, 4))
grid = ImageGrid(fig, 111, nrows_ncols=(1,4), axes_pad=0.05)
for i in range(img.size()[0]):
    ax = grid[i]
    imshow(ax, img[i])

In [18]:
use_gpu = torch.cuda.is_available()
resnet = models.resnet50(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False
    
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)
if use_gpu:
    resnet.cuda()
inputs, labels = next(iter(train_dl))
if use_gpu:
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
else:
    inputs, labels = Variable(inputs), Variable(labels)
outputs = resnet(inputs)
outputs.size()

torch.Size([4, 14951])

In [ ]:
_, preds = torch.max(outputs.data, 1)

In [ ]:
preds

In [ ]:
labels

In [ ]:
torch.sum(preds == labels.data)

In [ ]:
NUM_EPOCHS = 1

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)

In [20]:
optimizer.zero_grad()

In [21]:
loss = criterion(outputs,labels)

In [22]:
loss.backward()

In [24]:
NUM_EPOCHS = 2

In [25]:
for epoch in range(NUM_EPOCHS):
    running_loss = 0
    for i, data in enumerate(train_dl, 0):
        inps, lbls = next(iter(train_dl))
        try:
            if use_gpu:
                resnet = resnet.cuda()
                inputs, labels = Variable(inps.cuda()), Variable(lbls.cuda())
            else:
                inputs, labels = Variable(inps), Variable(lbls)
            optimizer.zero_grad()    
            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            if i % 100 == 0:
                correct = 0
                total = 0
                for data in valid_dl:
                    images, labels = data
                    outputs = resnet(Variable(images.cuda()))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted.cpu() == labels).sum()
                accuracy = 100 * correct / total
                print("Loss: {}, Running Loss: {}, Correct: {}, Total: {}, Valid Accuracy: {}".format(
                    loss.data[0], running_loss, correct, total, accuracy))
        except Exeption as error:
            print(inps.size, lbls.size)
        

Loss: 9.585684776306152, Running Loss: 9.585684776306152, Correct: 0, Total: 245, Valid Accuracy: 0.0
Loss: 10.00446891784668, Running Loss: 913.9179682731628, Correct: 10, Total: 245, Valid Accuracy: 4.081632653061225
Loss: 8.50673770904541, Running Loss: 1723.718605041504, Correct: 16, Total: 245, Valid Accuracy: 6.530612244897959
Loss: 8.883787155151367, Running Loss: 8.883787155151367, Correct: 15, Total: 245, Valid Accuracy: 6.122448979591836
Loss: 7.571465969085693, Running Loss: 766.7230825424194, Correct: 19, Total: 245, Valid Accuracy: 7.755102040816326
Loss: 8.526094436645508, Running Loss: 1500.599160194397, Correct: 21, Total: 245, Valid Accuracy: 8.571428571428571


In [ ]:
_, predicted = torch.max(outputs.data, 1)

In [ ]:
outputs = resnet(Variable(images.cuda()))

In [ ]:
_, predicted = torch.max(outputs.data, 1)

In [29]:
torch.save(resnet, "resnet_trained.pkl")

In [28]:
! ls

Cifar Practice.ipynb			  imaterialist_full.pkl
data					  iMaterialist.ipynb
DataLoader.ipynb			  imaterialist.py
df1.pkl					  pretrained-pytorch-models
df_full.pkl				  pretrained-pytorch-models.zip
Download Script.ipynb			  resnet_trained.pkl
Google Landmark Dataset.ipynb		  Simple Exploration of Data.ipynb
google-landmark-test-set.ipynb		  Torch Vision Test.ipynb
google-landmark-torch-vision-Copy1.ipynb  train_df.pkl
google-landmark-torch-vision.ipynb	  Train.ipynb
iDataSample.pkl				  wget-log
